In [1]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as mc
import warnings
import numpy as np
from tqdm import tqdm
import spacy

In [2]:
def Add_subset(k, scores, test_idx, pred, X, y, thresh):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > thresh:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]
        
    y_k = y["train_add"][top_k_index]
    X_k = X["train_add"][top_k_index]
    y_added = np.concatenate((y["train"],y_k))
    X_added = np.concatenate((X["train"],X_k), axis=0)

    prediction = -np.sum(scores[test_idx][top_k_index])
    #print("prediction", prediction)

    return X_k, y_k, prediction, top_k_index


def loss_gradient(X, y, model):
    F_train = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
    error_train = model.predict_proba(X)[:, 1] - y
    gradient_train = F_train * error_train[:, None]
    return gradient_train

In [3]:
def approximate_k(test_idx, pred, delta_pred, y, thresh):
    old = pred[test_idx].item()
    
    if pred[test_idx] > thresh:
        top_k_index = np.flip(delta_pred[test_idx].argsort())
    else:
        top_k_index = delta_pred[test_idx].argsort()
    
    for k in range(1, y["train"].shape[0]):
        change = -np.sum(delta_pred[test_idx][top_k_index[:k]])
        
        if old > thresh and old + change < thresh:
            return k
        elif old < thresh and old + change > thresh:
            return k
        
    return None

In [4]:
def new_train(k, dev_index, scores, l2, X, model, pred, y, thresh):
    X_k, y_k, prediction, top_k_index = Add_subset(k, scores, dev_index, pred, X, y, thresh)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None, None, None, None, None, None
    
    model.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model.predict_proba(test_point)[0][1]
    change = -(model.predict_proba(test_point)[0][1] - new)
    #change = model_k.predict_proba(test_point)[0][1]-model.predict_proba(test_point)[0][1]
    flip = (model.predict(test_point) == model.predict(test_point))
    
    """
    print("change    ", change)
    print("old       ", model.predict_proba(test_point)[0][1])
    print()
    """
    error = np.abs((change - prediction)/prediction)
    return change, flip, prediction, new, error, top_k_index

In [5]:
def IP(X, y, l2, dataname, thresh, model, pred, modi=None):
    gradient_train_add = loss_gradient(X["train_add"], y["train_add"], model)

    w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
    F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))],
                             axis=1)  # Concatenating one to calculate the gradient with respect to intercept
    F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

    # error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
    # error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

    # gradient_train = F_train * error_train[:, None]
    # gradient_dev = F_dev * error_dev[:, None]

    probs = model.predict_proba(X["train"])[:, 1]
    hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / \
              X["train"].shape[0]
    inverse_hessian = np.linalg.inv(hessian)
    eps = 1 / X["train"].shape[0]
    delta_k = -eps * inverse_hessian @ (- gradient_train_add).T
    grad_f = F_dev * (pred * (1 - pred))
    delta_pred = grad_f @ delta_k
    return delta_pred
    


    


In [6]:
import spacy
sst_dataset = {}
for split in ["train", "dev", "test"]:
    URL = f"https://raw.githubusercontent.com/successar/instance_attributions_NLP/master/Datasets/SST/data/{split}.jsonl"
    import urllib.request, json
    with urllib.request.urlopen(URL) as url:
        data = url.read().decode()
        data = [json.loads(line) for line in data.strip().split("\n")]
        sst_dataset[split] = data


nlp = spacy.load('en_core_web_sm')

X, y = {}, {}
for split in ["train", "dev"]:
    X[split] = np.array([nlp(example["document"]).vector for example in tqdm(sst_dataset[split])])
    y[split] = np.array([example["label"] for example in sst_dataset[split]])

100%|██████████| 872/872 [00:05<00:00, 167.83it/s]


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_train_add, y_train, y_train_add = train_test_split(
    X["train"], y["train"], test_size=0.4, random_state=42)

In [8]:
X["train"] = X_train
X["train_add"] = X_train_add
y["train"] = y_train
y["train_add"]= y_train_add
X["train"].shape, X["train_add"].shape, X["dev"].shape

((4152, 96), (2768, 96), (872, 96))

In [9]:
thresh = 0.5
l2 = 10
model = LogisticRegression(penalty='l2', C=1/l2, solver='saga',warm_start=True)
model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))

In [10]:
delta_pred = IP(X, y, l2, "SST", thresh, model, pred)
delta_pred.shape

(872, 2768)

In [11]:
# Loop over all dev points:
appro_ks = []
new_predictions = []
flip_list = []
for test_idx in tqdm(range(X["dev"].shape[0])):
    appro_k = approximate_k(test_idx, pred, delta_pred, y, thresh)
    if appro_k != None:
        change, _, prediction,new_prediction, error, top_k_index = new_train(appro_k, test_idx, delta_pred, l2, X, model, pred, y, thresh)
        print(test_idx, appro_k, pred[test_idx], new_prediction)
        appro_ks.append(appro_k)
        new_predictions.append(new_prediction)
        flip_list.append(top_k_index)
    else:
        appro_ks.append(None)
        new_predictions.append(None)
        flip_list.append(None)

appro_ks= np.array(appro_ks)
new_predictions=np.array(new_predictions)
# flip_list = np.array(flip_list)
# np.save("./results/" + "appro_ks_IP" + "_alg1_" + dataname  + str(l2) + ".npy", appro_ks)
# np.save("./results/" + "new_predictions" + "_alg1_" + dataname + str(l2) + ".npy", new_predictions)
# np.save("./results/" + "old_predictions" + "_alg1_" + dataname + str(l2) + ".npy", pred)
# np.save("./results/" + "flip_list" + "_alg1_" + dataname + str(l2) + ".npy", flip_list)

  0%|          | 0/872 [00:00<?, ?it/s]/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
  3%|▎         | 23/872 [00:00<00:03, 226.86it/s]/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not con

0 20 [0.5509179] 0.23607384
1 34 [0.40708712] 0.8511833
2 72 [0.6836342] 0.026963012
3 4 [0.49117252] 0.52820253
4 42 [0.6547108] 0.074020796
5 92 [0.63173795] 0.11043312
6 6 [0.484108] 0.5958128
7 9 [0.48168048] 0.8066516
8 161 [0.6890086] 0.05747488
9 29 [0.5548766] 0.39529526
10 25 [0.4435639] 0.77873194
11 8 [0.5202813] 0.35272473
12 73 [0.6395619] 0.061469458
13 45 [0.37409356] 0.94439125
14 127 [0.6812974] 0.052479442
15 19 [0.53945047] 0.2141494
16 43 [0.5639916] 0.1365217
17 15 [0.45675865] None
18 173 [0.69866514] 0.03239461
19 30 [0.56355983] 0.21295026
20 44 [0.5752861] 0.2331422
21 40 [0.6803442] 0.05015972
22 112 [0.65148365] 0.07550993
23 112 [0.69692945] 0.02965681
24 3 [0.48383203] 0.33698374
25 11 [0.5441342] 0.19649531
26 94 [0.62871855] 0.08128653
27 110 [0.32732755] 0.9107533
28 22 [0.5574646] 0.16651124
29 14 [0.53091234] 0.2708724
30 112 [0.67683524] 0.060638648
31 58 [0.38949615] 0.8606737
32 22 [0.5504116] 0.1831223
33 45 [0.58985686] 0.11287512
34 5 [0.5132863]

  6%|▌         | 52/872 [00:00<00:03, 257.09it/s]/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
  9%|▉         | 78/872 [00:00<00:03, 242.84it/s]/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ d

51 65 [0.6296311] 0.07350321
52 163 [0.6934624] 0.04695206
53 30 [0.5782436] 0.20339026
54 78 [0.61627805] 0.05982234
55 22 [0.5495632] 0.3518882
56 7 [0.5244039] 0.3992227
57 61 [0.38089994] 0.8948031
58 69 [0.3218399] 0.923836
59 34 [0.41967943] 0.93134534
60 21 [0.541312] 0.09392139
61 112 [0.65289956] 0.10600053
62 65 [0.6444478] 0.08612042
63 57 [0.60406274] 0.11171479
64 12 [0.546608] 0.2404296
65 11 [0.51781946] 0.31414303
66 116 [0.62637115] 0.057322454
67 176 [0.7404305] 0.026491715
68 68 [0.6054877] 0.12769097
69 24 [0.42987174] 0.8912959
70 36 [0.4423776] 0.88641465
71 52 [0.37785205] 0.91667026
72 73 [0.6557302] 0.046603248
73 52 [0.57818687] 0.14066175
74 3 [0.5082837] None
75 65 [0.40364125] 0.868941
76 101 [0.665291] 0.055766836
77 1 [0.49911857] None
78 15 [0.47279027] 0.78608465
79 46 [0.5985158] 0.163559
80 8 [0.5477181] 0.12542789
81 16 [0.5502247] 0.085509114
82 38 [0.3529809] 0.9317658
83 8 [0.5252721] 0.55291855
84 47 [0.59397006] 0.18307318
85 111 [0.6649714] 0.0

/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linea

...

762 27 [0.55760896] 0.17364971
763 79 [0.30475608] 0.9540682
764 63 [0.6354386] 0.061017927
765 32 [0.55391175] 0.397696
766 41 [0.6486804] 0.064243846
767 13 [0.4521294] 0.89600927
768 82 [0.63169503] 0.106182545
769 17 [0.46181098] 0.79092735
770 17 [0.4495179] 0.85091865
771 3 [0.50984055] 0.33869198
772 24 [0.44330627] 0.79628146
773 101 [0.6648565] 0.06133768
774 42 [0.38985157] 0.9039337
775 55 [0.34118658] 0.88786906
776 97 [0.6338315] 0.08113212
777 2 [0.50529516] 0.4764626
778 15 [0.5379583] 0.38333815
779 100 [0.35306618] 0.8963701
780 4 [0.50945276] 0.41499817
781 36 [0.578228] 0.18695521
782 51 [0.59971654] 0.15675944
783 6 [0.5177216] 0.7517573
784 18 [0.46370628] 0.7700561
785 37 [0.40695792] 0.9140491
786 13 [0.46729422] 0.85830784
787 13 [0.54338855] 0.18431991
788 5 [0.5261692] 0.1815935
789 97 [0.34905764] 0.95004493
790 46 [0.37981725] 0.88584554
791 30 [0.4181723] 0.73025554
792 114 [0.30469024] 0.9300911
793 4 [0.48876864] 0.5761387
794 30 [0.56898624] 0.35837334
7

/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linea

819 34 [0.57056254] 0.1515238
820 52 [0.3971405] 0.86501414
821 7 [0.48051193] 0.60036284
822 40 [0.5968163] 0.10866005
823 106 [0.26224294] 0.97674495
824 13 [0.47306937] 0.8445809
825 14 [0.5236346] 0.51391417
826 4 [0.48093235] 0.8053637
827 76 [0.3707889] 0.9049733
828 115 [0.7207189] 0.03525137
829 21 [0.55326897] 0.24762115
830 26 [0.6476711] 0.09092302
831 43 [0.38733575] 0.9324196
832 14 [0.4702725] 0.5614425
833 57 [0.62894976] 0.11847074
834 20 [0.5740479] 0.20983711
835 17 [0.45477152] 0.77891415
836 479 [0.12365636] 0.99964535
837 6 [0.48472655] 0.853882
838 12 [0.47304466] 0.702527
839 27 [0.406788] 0.86744773
840 5 [0.54287887] None
841 27 [0.41354173] 0.8372244
842 117 [0.3071623] 0.930484
843 64 [0.28404498] 0.98509014
844 7 [0.4732421] 0.89528507
845 43 [0.55489606] 0.17281999
846 29 [0.5648009] 0.24940093
847 2 [0.505845] None
848 36 [0.42216545] 0.8111851
849 200 [0.26270947] 0.9757721
850 30 [0.5733596] 0.10964876
851 75 [0.30751362] 0.95844054
852 4 [0.5137675] 0.4

/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/if/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
100%|██████████| 872/872 [00:03<00:00, 250.99it/s]

868 8 [0.47568718] 0.6451312
869 23 [0.45380282] 0.8009489
870 34 [0.58116955] 0.11251625
871 5 [0.48611158] 0.6050083


In [12]:
new_predictions = new_predictions.reshape((872, 1))
eval = []
num = 0
for i in range(872):
    if new_predictions[i] != None:
        eval.append((new_predictions[i] > 0.5) != (pred[i] > 0.5))
        num += 1
np.sum(eval) / num

np.float64(0.9623115577889447)